In [1]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as sm

# 1. Read microdata

In [2]:
microdata = pd.read_excel("assets/excel/esa_microdata.xlsx", index_col=[0])

In [3]:
display(microdata)

,stromverbrauch_jahr_kwh,strompreis_cent_kwh,einkommen_gesamt_jahr_eur,anz_pers
0,1172.873881,33.18,19909.542969,2
1,1296.848970,27.70,21712.037109,2
2,1232.803083,31.65,22188.449219,2
3,1314.827897,23.98,19398.164062,2
4,1337.919699,26.64,22812.015625,2
...,...,...,...,...
14092,2127.427962,28.92,13863.910156,4
14093,2307.423594,28.06,26886.554688,4
14094,1157.814212,35.89,22467.421875,2
14095,1137.222560,31.95,15723.962891,2


# 2. Divide into households groups by household income

In [4]:
# The total number of households is equal to the number of rows in the microdata set
number_of_households = len(microdata.index)

In [5]:
# Calculate the median income for the calculation of poverty indicators
median_income = microdata["einkommen_gesamt_jahr_eur"].median()

### Low income households

In [6]:
low_income_threshold = 1/3  # Households in the bottom third of the income distribution"

# Sort the values by income and select the household income that is located at the threshold
low_income_threshold_income_eur = microdata.sort_values(by="einkommen_gesamt_jahr_eur").iloc[int(low_income_threshold*number_of_households)]["einkommen_gesamt_jahr_eur"]

# 3. Calculate price and income elasticities of demand

#### Define regression equation

In [7]:
regression_formula = "np.log(stromverbrauch_jahr_kwh) ~ np.log(strompreis_cent_kwh) + np.log(einkommen_gesamt_jahr_eur) + anz_pers"

#### Create regression model

In [8]:
ols_model = sm.ols(formula=regression_formula, data=microdata)

#### Fit model to data

In [9]:
ols_model_results = ols_model.fit()

#### Show regression statistics

In [10]:
display(ols_model_results.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                                   OLS Regression Results                                  
===========================================================================================
Dep. Variable:     np.log(stromverbrauch_jahr_kwh)   R-squared:                       0.778
Model:                                         OLS   Adj. R-squared:                  0.778
Method:                              Least Squares   F-statistic:                 1.650e+04
Date:                             Tue, 26 Jul 2022   Prob (F-statistic):               0.00
Time:                                     19:04:17   Log-Likelihood:                 2338.4
No. Observations:                            14097   AIC:                            -4669.
Df Residuals:                                14093   BIC:                            -4639.
Df Model:                                        3                                         
Covariance Type:                         nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             5.5924      0.045    123.411      0.000       5.504       5.681
np.log(strompreis_cent_kwh)          -0.3859      0.012    -32.357      0.000      -0.409      -0.363
np.log(einkommen_gesamt_jahr_eur)     0.2571      0.002    132.550      0.000       0.253       0.261
anz_pers                              0.2182      0.001    180.030      0.000       0.216       0.221
==============================================================================
Omnibus:                     4688.283   Durbin-Watson:                   0.953
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            20467.459
Skew:                           1.582   Prob(JB):                         0.00
Kurtosis:                       7.984   Cond. No.                         301.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# 4. Calculate impact of relief measures

In [ ]:
price_discount_cent_kwh = -5 # €-ct/kWh

In [ ]:
electricity_compensation_eur = 350  # €/a

### For the case of a general price discount

In [ ]:
def calc_consumption_change_price_discount(old_consumption, price_elasticity, original_price, price_discount_cent_kwh):
    new_price = original_price + price_discount_cent_kwh
    return np.exp(price_elasticity*(np.log(new_price)-np.log(original_price))+np.log(old_consumption))

In [ ]:
price_elasticity = -0.3859
microdata["stromverbrauch_jahr_preisrabatt_kwh"] = calc_consumption_change_price_discount(microdata["stromverbrauch_jahr_kwh"], price_elasticity, microdata["strompreis_cent_kwh"], price_discount_cent_kwh)

### For the case of an electricity cost compensation for low-income households

In [ ]:
def calc_consumption_change_compensation_payment(old_consumption, income_elasticity, old_income, new_income):
    return np.exp(income_elasticity*(np.log(new_income.astype(float))-np.log(old_income.astype(float)))+np.log(old_consumption.astype(float)))

In [ ]:
microdata["einkommen_gesamt_jahr_mit_zuschuss_eur"] = microdata["einkommen_gesamt_jahr_eur"]
microdata.loc[microdata["einkommen_gesamt_jahr_eur"] <= low_income_threshold_income_eur, "einkommen_gesamt_jahr_mit_zuschuss_eur"] += electricity_compensation_eur

In [ ]:
income_elasticity =  0.2571
microdata["stromverbrauch_jahr_mit_zuschuss_kwh"] = calc_consumption_change_compensation_payment(microdata["stromverbrauch_jahr_kwh"], income_elasticity, microdata["einkommen_gesamt_jahr_eur"], microdata["einkommen_gesamt_jahr_mit_zuschuss_eur"])

# 5. Analyze impact of relief measures

### Without relief measures

In [ ]:
microdata["anteil_stromkosten"] = microdata["stromverbrauch_jahr_kwh"] * (microdata["strompreis_cent_kwh"]/100) / microdata["einkommen_gesamt_jahr_eur"]

In [ ]:
print("Durchschnittl. Anteil der Stromausgaben am Einkommen:",round(microdata["anteil_stromkosten"].mean()*100,2), "%")

In [ ]:
print("Gesamte Stromnachfrage:",round((microdata["stromverbrauch_jahr_kwh"]/1e6).sum(),2), "GWh")

In [ ]:
print("Armutsgefährdete Haushalte:",round(len(microdata.loc[microdata["einkommen_gesamt_jahr_eur"] <= 0.6*median_income].index)/number_of_households*100,2), "%")

### With price discount

In [ ]:
microdata["anteil_stromkosten_preisrabatt"] = microdata["stromverbrauch_jahr_preisrabatt_kwh"] * ((microdata["strompreis_cent_kwh"]+price_discount_cent_kwh)/100) / microdata["einkommen_gesamt_jahr_eur"]

In [ ]:
microdata["einkommen_gesamt_jahr_preisrabatt_eur"] = microdata["einkommen_gesamt_jahr_eur"] + microdata["stromverbrauch_jahr_kwh"] * microdata["strompreis_cent_kwh"]/100 - microdata["stromverbrauch_jahr_preisrabatt_kwh"] * (microdata["strompreis_cent_kwh"]+price_discount_cent_kwh)/100

In [ ]:
print("Durchschnittl. Anteil der Stromausgaben am Einkommen:",round(microdata["anteil_stromkosten_preisrabatt"].mean()*100,2), "%")

In [ ]:
print("Gesamte Stromnachfrage:",round((microdata["stromverbrauch_jahr_preisrabatt_kwh"]/1e6).sum(),2), "GWh")

In [ ]:
print("Armutsgefährdete Haushalte:",round(len(microdata.loc[microdata["einkommen_gesamt_jahr_preisrabatt_eur"] <= 0.6*median_income].index)/number_of_households*100,2), "%")

In [ ]:
print("Gesamtkosten der Maßnahme:",round(((microdata["stromverbrauch_jahr_preisrabatt_kwh"]*-price_discount_cent_kwh)/100/1e6).sum(),2), "Mil. €")

### With compensation payment for low-income households

In [ ]:
microdata["anteil_stromkosten_mit_zuschuss"] = microdata["stromverbrauch_jahr_mit_zuschuss_kwh"] * (microdata["strompreis_cent_kwh"]/100) / microdata["einkommen_gesamt_jahr_mit_zuschuss_eur"]

In [ ]:
print("Durchschnittl. Anteil der Stromausgaben am Einkommen:",round(microdata["anteil_stromkosten_mit_zuschuss"].mean()*100,2), "%")

In [ ]:
print("Gesamte Stromnachfrage:",round((microdata["stromverbrauch_jahr_mit_zuschuss_kwh"]/1e6).sum(),2), "GWh")

In [ ]:
print("Armutsgefährdete Haushalte:",round(len(microdata.loc[microdata["einkommen_gesamt_jahr_mit_zuschuss_eur"] <= 0.6*median_income].index)/number_of_households*100,2), "%")

In [ ]:
print("Gesamtkosten der Maßnahme:",round((1/3)*number_of_households*electricity_compensation_eur/1e6,2), "Mil. €")